# Mavlink  MavCom
> Mavlink  base class for Client and server.
> Theserver is on the companion computer and the client is on the ground station PC.

[https://mavlink.io/en/mavgen_python/](https://mavlink.io/en/mavgen_python/)
[https://www.ardusub.com/developers/pymavlink.html](https://www.ardusub.com/developers/pymavlink.html)

https://mavlink.io/en/messages/common.html
https://mavlink.io/en/messages/common.html#MAV_TYPE



In [ ]:
#| default_exp mavlink.test_mavcom 

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and 
# skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

In [ ]:
#| export
import time, os, sys

from mavcom.logging import logging
from mavcom.utils.general import LeakyQueue

# os.environ['MAVLINK20'] == '1' should be placed in mavcom.__init__.py
assert os.environ[
           'MAVLINK20'] == '1', "Set the environment variable before from pymavlink import mavutil  library is imported"

# logging.getLogger("uav").setLevel(logging.DEBUG)
# logging.root.setLevel(logging.INFO)
import threading
import queue
import typing as typ
from pathlib import Path
from inspect import currentframe, getframeinfo
from pymavlink import mavutil
from mavcom.mavlink import Component, MAVCom
from mavcom.mavlink.mavcom import *
from fastcore.test import *
# from mavcom.imports import *   # TODO why is this relative import on nbdev_export?


In [ ]:
# logging.getLogger("uav").setLevel(logging.INFO)

In [ ]:
#| hide
from fastcore.utils import *
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
show_doc(get_linenumber)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/mavcom.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### get_linenumber

>      get_linenumber ()

In [ ]:

MAV_SYSTEM_GCS_CLIENT = 200  # GCS type client (TODO its not clear if this is correct,  255 = GCS)
MAV_TYPE_GCS = mavutil.mavlink.MAV_TYPE_GCS
MAV_SYSTEM_VEHICLE = 111  # 1 = vehicle
MAV_TYPE_CAMERA = mavutil.mavlink.MAV_TYPE_CAMERA
MAV_COMP_ID_CAMERA = mavutil.mavlink.MAV_COMP_ID_CAMERA
MAV_COMP_ID_USER1 = mavutil.mavlink.MAV_COMP_ID_USER1

In [ ]:
show_doc(Component)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/mavcom.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### BaseComponent

>      BaseComponent (mav_connection, source_component, mav_type, debug)

Create a mavlink Component with an ID  for MAV_COMPONENT

|    | **Details** |
| -- | ----------- |
| mav_connection | MavLinkBase connection |
| source_component | used for component indication |
| mav_type | used for heartbeat MAV_TYPE indication |
| debug | logging level |

In [ ]:
show_doc(MAVCom)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/mavcom.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### MAVCom

>      MAVCom (connection_string, baudrate=57600, source_system=111,
>              debug=False)

Mavlink Base to set up a mavlink_connection for send and receive messages to and from a remote system.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| connection_string |  |  | "udpin:localhost:14550" |
| baudrate | int | 57600 | baud rate of the serial port |
| source_system | int | 111 | remote or air uav system   1 = vehicle |
| debug | bool | False | logging level |

In [ ]:
# | hide
# assert False, "Stop here"

In [ ]:
# show_doc(MavLinkBase)

#### Test locally using UDP ports

#### Starting a client and server
 > on the same machine using UDP ports `14445`  with `server_system_ID=111, client_system_ID=222`

In [ ]:

#|exports
def test_MAVCom():
    """Test MAVCom with a client and server on the same machine using UDP ports `14445`  with `server_system_ID=111, client_system_ID=222`"""
    with MAVCom("udpin:localhost:14445", source_system=111, debug=False) as client:
        with MAVCom("udpout:localhost:14445", source_system=222, debug=False) as server:
            server.add_component(Component(server, mav_type=MAV_TYPE_CAMERA, source_component = 22, debug=False))
            client.add_component(Component(client, mav_type=MAV_TYPE_GCS, source_component = 11, debug=False))
    
            MAX_PINGS = 4
            client.component[11].send_ping(222, 22, max_pings=MAX_PINGS)
            time.sleep(0.5)
    
    print(f"{server.source_system = };  {server.message_cnts = }")
    print(f"{client.source_system = };  {client.message_cnts = }")
    
    test_eq(server.message_cnts[111]['PING'], MAX_PINGS)
    test_eq(server.message_cnts[111]['HEARTBEAT']>0, True) 
    test_eq(client.message_cnts[222]['PING'], MAX_PINGS)
    test_eq(client.message_cnts[222]['HEARTBEAT']>0, True)
    
test_MAVCom()

INFO   | mavcom.MAVCom      | 53.341 |  mavcom.py:393 | Thread-7 (listen)  | MAVLink Mav2: True, source_system: 111
INFO   | mavcom.MAVCom      | 53.443 |  mavcom.py:393 | Thread-8 (listen)  | MAVLink Mav2: True, source_system: 222
INFO   | mavcom.MAVCom      | 53.443 |  mavcom.py:441 | MainThread         | MAVCom  closed
INFO   | mavcom.MAVCom      | 54.343 |  mavcom.py:441 | MainThread         | MAVCom  closed


AttributeError: 'NoneType' object has no attribute 'source_system'

In [ ]:
#| Hide
# assert False, "Stop here"

In [ ]:
show_doc(MAVCom.add_component)

In [ ]:
show_doc(MAVCom.close)

This will show on wireshark as follows:
> Using the display filte string `not icmp  && udp.port eq 14445 && mavlink_proto`

![](images/wireshark_udp:14445.png)

For debugging help see [http://localhost:3000/tutorials/mavlink_doc&debug.html](http://localhost:3000/tutorials/mavlink_doc&debug.html)
 and [http://localhost:3000/tutorials/mavlink_doc&debug.html#debugging](http://localhost:3000/tutorials/mavlink_doc&debug.html#debugging)

In [ ]:
#| hide
# from nbdev import nbdev_export
# nbdev_export()